In [2]:
from sumpy.recurrence import _make_sympy_vec, get_processed_and_shifted_recurrence

from sumpy.expansion.diff_op import (
    laplacian,
    make_identity_diff_op,
)

from sumpy.recurrence import get_recurrence

import sympy as sp
import numpy as np

In [3]:
w = make_identity_diff_op(2)
laplace2d = laplacian(w)
n_init, order, r = get_processed_and_shifted_recurrence(laplace2d)

In [4]:
def scale_recurrence(r):
    #We want to subsitute s(i) r^i_{ct} = g(i)
    g = sp.Function("g")
    s = sp.Function("s")
    n = sp.symbols("n")
    rct = sp.symbols("r_{ct}")

    r_new = r*rct**n
    for i in range(order):
        r_new = r_new.subs(s(n-i),g(n-i)/(rct**(n-i)))

    return r_new

In [5]:
r_new = scale_recurrence(r)

In [6]:
max_abs = .0000001
var = _make_sympy_vec("x", 2)
rct = sp.symbols("r_{ct}")
g = sp.Function("g")
n = sp.symbols("n")

In [7]:
r_new

(-1)**(n + 1)*r_{ct}**n*((-1)**(n - 3)*r_{ct}**(3 - n)*(n + (n - 2)**3 - 2*(n - 2)**2 - 2)*g(n - 3)/(x0**3 + x0*x1**2) + (-1)**(n - 2)*r_{ct}**(2 - n)*(-n + 3*(n - 2)**2 + 2)*g(n - 2)/(x0**2 + x1**2) + (-1)**(n - 1)*r_{ct}**(1 - n)*(3*x0**2*(n - 2) + x0**2 + x1**2*(n - 2) - x1**2)*g(n - 1)/(x0**3 + x0*x1**2))

In [8]:
def compute_derivatives(p):
    var = _make_sympy_vec("x", 2)
    var_t = _make_sympy_vec("t", 2)
    g_x_y = sp.log(sp.sqrt((var[0]-var_t[0])**2 + (var[1]-var_t[1])**2))
    derivs = [sp.diff(g_x_y,
                        var_t[0], i).subs(var_t[0], 0).subs(var_t[1], 0)
                        for i in range(p)]
    return derivs
derivs = compute_derivatives(15)

In [ ]:
def evaluate_recurrence(coord_dict, rct_val, recur, p):
    subs_dict = {}
    subs_dict[g(0)] = derivs[0].subs(coord_dict).subs(rct, rct_val)
    subs_dict[g(1)] = derivs[1].subs(coord_dict).subs(rct, rct_val) * rct_val
    var = _make_sympy_vec("x", 2)
    for i in range(2, p):
        subs_dict[g(i)] = get_recurrence(recur.subs(rct, rct_val), i).subs(subs_dict).subs(coord_dict)
        print(get_recurrence(recur.subs(rct, 1),i))
    return np.array(list(subs_dict.values()))

In [97]:
def evaluate_recurrence_lamb(coord_dict, rct_val, recur, p):
    subs_dict = {}
    subs_dict[g(-2)] = 0
    subs_dict[g(-1)] = 0
    subs_dict[g(0)] = derivs[0].subs(coord_dict).subs(rct, rct_val)
    subs_dict[g(1)] = derivs[1].subs(coord_dict).subs(rct, rct_val) * rct_val
    var = _make_sympy_vec("x", 2)
    for i in range(2, p):
        exp = get_recurrence(recur.subs(rct, rct_val), i)
        f = sp.lambdify([var[0], var[1], g(i-1), g(i-2), g(i-3)], exp)
        subs_dict[g(i)] = f(coord_dict[var[0]], coord_dict[var[1]], subs_dict[g(i-1)],
                            subs_dict[g(i-2)], subs_dict[g(i-3)])
    subs_dict.pop(g(-2))
    subs_dict.pop(g(-1))
    return np.array(list(subs_dict.values()))

In [98]:
def evaluate_true(coord_dict, rct_val, p):
    retMe = []
    for i in range(p):
        exp = (derivs[i]*rct_val**i)
        f = sp.lambdify(var, exp)
        retMe.append(f(coord_dict[var[0]], coord_dict[var[1]]))
    return np.array(retMe)

In [103]:
def compute_error(pw):
    x_coord = 10**-pw * np.random.rand()
    y_coord = 10**-pw * np.random.rand()
    var = _make_sympy_vec("x", 2)
    coord_dict = {var[0]: x_coord, var[1]: y_coord}

    rct_val = 1
    exp = evaluate_recurrence_lamb(coord_dict, rct_val, r_new, 14)
    true = evaluate_true(coord_dict, rct_val, 14)
    print(exp)
    print(true)
    return np.abs(exp-true)/np.abs(true)

In [106]:
compute_error(7)

[-16.4793912653345 -7446016.24208662 95090046267340.2 5.89957479007560e+21
 1.46052167481843e+29 -5.88202094275608e+36 -1.03964211722783e+45
 -5.65475868362143e+52 3.09919194622245e+60 1.02148424529142e+69
 9.09458487415835e+76 -5.39245944612498e+84 -2.94394795078325e+93
 -3.79481490540789e+101]
[-1.64793913e+001 -7.44601624e+006  9.50900463e+013  5.89957479e+021
  1.46052167e+029 -5.88202094e+036 -1.03964212e+045 -5.65475868e+052
  3.09919195e+060  1.02148425e+069  9.09458487e+076 -5.39245945e+084
 -2.94394795e+093 -3.79481491e+101]


array([0, 1.25076624108262e-16, 1.64317934561428e-16,
       1.77737555215664e-16, 1.08406247664505e-15, 2.20783094012695e-15,
       1.06690009650372e-15, 7.52203555358031e-16, 1.09374099087612e-14,
       6.56365286316817e-15, 1.83759411688817e-15, 3.15172520976218e-14,
       2.34420194715335e-14, 2.98146389490554e-14], dtype=object)

In [68]:
pw = 8
x0 = 1
x1 = 10**(-pw)
g_1 = -5000000000.00000
g_2 = (x0**2 - x1**2)*g_1/(x0**3 + x0*x1**2)
g_2

-0.0

In [72]:
derivs[7]

-720*x0*(64*x0**6/(x0**2 + x1**2)**3 - 112*x0**4/(x0**2 + x1**2)**2 + 56*x0**2/(x0**2 + x1**2) - 7)/(x0**2 + x1**2)**4

In [71]:
derivs[2].subs(var[0], 10**(-8)).subs(var[1], 10**(-8))

0.555111512312578

In [51]:
g_2

-0.0

In [14]:
'''
x_plot = [i for i in range(len(compute_error(0)))]
for i in range(1, 4):
    plt.semilogy(x_plot, compute_error(i), label=str(10**(-i)))
plt.xlabel("order of derivative being computed")
plt.ylabel("absolute error")
plt.title("recurrence error vs order for different source-locations")
plt.legend(title='ratio of x_{coord_src}/y_{coord_src}')
plt.show()
'''

'\nx_plot = [i for i in range(len(compute_error(0)))]\nfor i in range(1, 4):\n    plt.semilogy(x_plot, compute_error(i), label=str(10**(-i)))\nplt.xlabel("order of derivative being computed")\nplt.ylabel("absolute error")\nplt.title("recurrence error vs order for different source-locations")\nplt.legend(title=\'ratio of x_{coord_src}/y_{coord_src}\')\nplt.show()\n'